In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [2]:
pd.set_option("display.max_columns", 100)

In [3]:
analysis_date = dt.date(2022,5,25)

In [66]:
analysis_date.isoformat()

'2022-05-25'

In [4]:
itp_id = 300

In [5]:
#route = 3479

In [124]:
sched = (tbl.views.gtfs_schedule_fact_daily_trips()
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
      >> collect()
)

In [125]:
sched

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,-276168916209754772,882890,3491,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,16,16,24240,25140,0.250000
1,4100255323842349957,5450854513072761642,883706,3501,300,0,40510,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,17,17,58560,59760,0.333333
2,4100255323842349957,4962984872484144342,882619,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,17,17,61380,63540,0.600000
3,4100255323842349957,-3228323852825060608,882569,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,17,17,25800,27240,0.400000
4,4100255323842349957,7255958991023188136,883703,3501,300,0,77710,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,17,17,56220,57300,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,4100255323842349957,8689229955998851824,881879,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,53,53,22680,25681,0.833611
1512,4100255323842349957,6546405398430871463,881913,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,43200,47040,1.066667
1513,4100255323842349957,-4531999280846723578,881921,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,36480,40320,1.066667
1514,4100255323842349957,3535108752229370095,881909,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,46560,50400,1.066667


In [126]:
sched.service_id.nunique()

6

In [127]:
sched.service_id.value_counts()

10       1409
40510      52
77710      52
92610       1
12010       1
96910       1
Name: service_id, dtype: int64

In [128]:
(sched
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> summarize(unique= _.trip_id.nunique())
      
)

,unique
0,1516


In [129]:
(sched
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> group_by(_.route_id) 
 >> summarize(n=_.trip_id.nunique()) >> arrange(-_.n))

,route_id,n
0,3479,180
4,3485,153
8,3489,153
2,3481,145
9,3490,107
18,3501,104
12,3493,88
1,3480,82
15,3496,76
5,3486,71


In [130]:
rt = query_sql(f'''
SELECT * FROM `cal-itp-data-infra-staging.nla_views.test_rt_trips`
WHERE date = '{analysis_date}'
''')

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [131]:
rt

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id
0,300,0,2022-05-25,880868,1300
1,300,0,2022-05-25,880880,1300
2,300,0,2022-05-25,880855,1300
3,300,0,2022-05-25,880788,1300
4,300,0,2022-05-25,880842,1300
...,...,...,...,...,...
1445,300,0,2022-05-25,881503,5319
1446,300,0,2022-05-25,883351,5320
1447,300,0,2022-05-25,881431,5320
1448,300,0,2022-05-25,881440,5320


In [132]:
sched>>distinct(_.trip_id)

,trip_id
0,882890
1,883706
2,882619
3,882569
4,883703
...,...
1511,881879
1512,881913
1513,881921
1514,881909


In [133]:
sched.sample()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
173,4100255323842349957,-1284954155374120969,883617,3501,300,0,40510,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,14,14,28860,30120,0.35


In [134]:
len(sched)

1516

In [136]:
test_join = sched>>inner_join(_, rt, on = 'trip_id')

In [137]:
len(test_join)

1450

In [138]:
sched_v_rt = (pd.merge(sched, rt, how='outer', on='trip_id', indicator='have_rt'))

In [139]:
sched_v_rt.have_rt.value_counts()

both          1450
left_only      168
right_only       0
Name: have_rt, dtype: int64

In [140]:
('{:.1%}'.format(1-(len(sched_v_rt>>filter(_.have_rt=='left_only')))/(len(sched_v_rt))))

'89.6%'

In [141]:
('{:.1%}'.format((len(sched_v_rt>>filter(_.have_rt=='left_only')))/(len(sched_v_rt))))

'10.4%'

In [142]:
('{:.1%}'.format((len(sched_v_rt>>filter(_.have_rt=='both')))/(len(sched_v_rt))))

'89.6%'

In [143]:
pct_not_ran = (
    (len(sched_v_rt>>filter(_.have_rt=='left_only')))
    /
    (len(sched_v_rt))
)
pct_not_ran = ('{:.1%}'.format(pct_not_ran))
pct_not_ran

'10.4%'

In [144]:
pct_ran = ((len(sched_v_rt>>filter(_.have_rt=='both')))/(len(sched_v_rt)))
pct_ran = ('{:.1%}'.format(pct_ran))
pct_ran

'89.6%'

In [62]:
display(Markdown(f"On {analysis_date.strftime('%B %d')}, **{pct_ran}** "
                "of scheduled trips produced GTFS RT data"))

On May 25, **92.5%** of scheduled trips produced GTFS RT data

In [23]:
# checking the length of the string to see if there are any incomplete/null `trip_ids`

In [24]:
rt>>filter(_.trip_id.isnull())

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id


In [25]:
rt['str_len'] = rt.trip_id.str.len()

In [26]:
rt>>arrange(-_.str_len)

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id,str_len
0,300,0,2022-05-25,880868,1300,6
1,300,0,2022-05-25,880880,1300,6
2,300,0,2022-05-25,880855,1300,6
3,300,0,2022-05-25,880788,1300,6
4,300,0,2022-05-25,880842,1300,6
...,...,...,...,...,...,...
1445,300,0,2022-05-25,881503,5319,6
1446,300,0,2022-05-25,883351,5320,6
1447,300,0,2022-05-25,881431,5320,6
1448,300,0,2022-05-25,881440,5320,6


In [27]:
## Putting into a function

In [153]:
def find_pct_ran_trips(itp_id, analysis_date):
    gtfs_daily = (
        tbl.views.gtfs_schedule_fact_daily_trips()
        >> filter(_.calitp_itp_id == itp_id)
        >> filter(_.service_date == analysis_date)
        >> filter(_.is_in_service == True)
        >> collect()
    )
    rt = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra-staging.nla_views.test_rt_trips`
        WHERE date = '{analysis_date}'
        """
    )
    
    rt['str_len'] = rt.trip_id.str.len()
    
    #joinging 
    join = (pd.merge(gtfs_daily, rt, how='outer', on='trip_id', indicator='have_rt'))
    
    return join

def get_pct_ran(join, date):
    ## eventually generate a dataframe with the columns (dates and pct_ran)
    day_pct_ran = {}
    day_pct_ran['date'] = date
    day_pct_ran['pct_trips_ran'] = ((len(join>>filter(_.have_rt=='both')))/(len(join)))
    
    pct_ran = pd.DataFrame([day_pct_ran])

    return pct_ran


In [154]:
join_day1 = find_pct_ran_trips(itp_id, analysis_date)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [155]:
join_day1.head()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,str_len,have_rt
0,4100255323842349957,-276168916209754772,882890,3491,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,16,16,24240,25140,0.250000,300.0,0.0,2022-05-25,1820,6.0,both
1,4100255323842349957,5450854513072761642,883706,3501,300,0,40510,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,17,17,58560,59760,0.333333,NaN,NaN,NaN,NaN,NaN,left_only
2,4100255323842349957,4962984872484144342,882619,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,17,17,61380,63540,0.600000,300.0,0.0,2022-05-25,1824,6.0,both
3,4100255323842349957,4962984872484144342,882619,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,17,17,61380,63540,0.600000,300.0,0.0,2022-05-25,2106,6.0,both
4,4100255323842349957,-3228323852825060608,882569,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,17,17,25800,27240,0.400000,300.0,0.0,2022-05-25,2114,6.0,both


In [156]:
pct_day1 = get_pct_ran(join_day1, analysis_date)
pct_day1

,date,pct_trips_ran
0,2022-05-25,0.896168


In [157]:
# add another date into the mix

In [158]:
analysis_date2 = dt.date(2022,5,26)

In [159]:
join_day2 = find_pct_ran_trips(itp_id, analysis_date2)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [160]:
join_day2.head()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,str_len,have_rt
0,4100255323842349957,235250936140600875,882521,3489,300,0,10,2022-05-26,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,52500,53880,0.383333,300.0,0.0,2022-05-26,3876,6.0,both
1,4100255323842349957,1191402648491880729,882557,3489,300,0,10,2022-05-26,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,27780,29400,0.450000,300.0,0.0,2022-05-26,2118,6.0,both
2,4100255323842349957,742245227560745521,883672,3501,300,0,40510,2022-05-26,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,17,17,32040,33060,0.283333,NaN,NaN,NaN,NaN,NaN,left_only
3,4100255323842349957,-306625641369824407,882568,3489,300,0,10,2022-05-26,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,17,17,25080,26520,0.400000,300.0,0.0,2022-05-26,2113,6.0,both
4,4100255323842349957,-827779873279239432,883690,3501,300,0,40510,2022-05-26,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,17,17,46080,46980,0.250000,NaN,NaN,NaN,NaN,NaN,left_only


In [161]:
join_day2.have_rt.value_counts()

both          1496
left_only      122
right_only       0
Name: have_rt, dtype: int64

In [162]:
pct_day2 = get_pct_ran(join_day2, analysis_date2)
pct_day2

,date,pct_trips_ran
0,2022-05-26,0.924598


In [163]:
pdList = [pct_day1, pct_day2]  # List of your dataframes
new_df = pd.concat(pdList)

In [165]:
new_df

,date,pct_trips_ran
0,2022-05-25,0.896168
0,2022-05-26,0.924598


In [174]:
# #another option for more dates... this works
# pdList2 = []
# pdList2.extend(value for name, value in locals().items() if name.startswith('pct_da'))
# new_df2 = pd.concat(pdList2)
# new_df2

In [234]:
date_list =['2022-05-25', '2022-05-26']

In [238]:
def get_pct_ran_df():
    pcts = []
    for date in date_list: 
        sched_rt_df = (find_pct_ran_trips(itp_id, date))
        pct_ran = (get_pct_ran(sched_rt_df, date))
        pcts.append(pct_ran)                                                    
        #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
    pcts = pd.concat(pcts)
    return pd.DataFrame(pcts)


In [239]:
final = get_pct_ran_df()

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [240]:
final

,date,pct_trips_ran
0,2022-05-25,0.896168
0,2022-05-26,0.924598


In [241]:
import altair as alt

In [244]:
chart = (
    alt.Chart(final)
    .mark_line()
    .encode(x="date", y="pct_trips_ran")
)

yrule = (
    alt.Chart().mark_rule(strokeDash=[12, 6], size=2).encode(y=alt.datum(1))
)


chart + yrule

alt.LayerChart(...)

In [237]:
# def get_pct_ran_list():
    
#     pdList2 = []
#     pdList2.extend(value for name, value in locals().items() if name.startswith('pct_da'))
#     new_df = pd.concat(pdList2)
#     return new_df

In [ ]:
## other queries

In [ ]:
# (tbl.views.gtfs_rt_fact_daily_feeds
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.date==analysis_date)
# )

In [ ]:
# trips =(tbl.views.gtfs_schedule_dim_trips()
#  >>filter(_.calitp_itp_id==itp_id) >> collect())


In [ ]:
#trips

In [ ]:
#pd.merge(df, trips, on="route_id")

In [ ]:
# (tbl.views.gtfs_schedule_fact_daily_trips()
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.service_date == analysis_date)
# )

In [ ]:
# (
#     tbl.views.gtfs_rt_fact_daily_feeds()
#     >> filter(_.calitp_itp_id == itp_id)
#     >> arrange(_.date ==analysis_date)
# )

In [ ]:
# airtable_organizations = (
#     tbl.airtable.california_transit_organizations()
#     >> select(_.itp_id, _.name, _.caltrans_district,
#               _.website, _.ntp_id, _.drmt_organization_name)
#     >> filter(_.itp_id == itp_id)
#     >> collect()
# )

In [ ]:
# airtable_organizations